# IEOR 242 Final Project
# Tab-Transformer

---

## Load Package

In [ ]:
import os

import pandas as pd
import numpy as np

import category_encoders as ce

from sklearn.preprocessing import label_binarize, StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    accuracy_score
)

from sklearn.cluster import KMeans, DBSCAN

import torch
import torch.nn.functional as F
from torch.nn.functional import softmax
from torch_geometric.data import Data
from torch_geometric.utils import mask_select
from torch_geometric.nn import GCNConv, GATConv

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

import os
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.utils import mask_select

import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from torch.nn.functional import softmax
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from sklearn.metrics import accuracy_score, f1_score
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.utils.class_weight import compute_class_weight
import optuna
from optuna.samplers import TPESampler
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
from sklearn.preprocessing import LabelEncoder

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from skopt import BayesSearchCV
from skopt.space import Integer, Real

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from scipy.sparse import issparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score

---

## Load Data

In [ ]:
directory = "/Users/yunxianghan/Desktop/jw/study/berkeley/ieor_242/final_project/data"
train_file_name = "collision_train_data.csv"
test_file_name = "collision_test_data.csv"

train_path = os.path.join(directory, train_file_name)
test_path = os.path.join(directory, test_file_name)

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
train_df.head()

,ACRS Report Type,Route Type,Road Name,Cross-Street Name,Weather,Surface Condition,Light,Driver Substance Abuse,Speed Limit,Vehicle Year,...,Driver At Fault,Vehicle Body Type,Vehicle Make,Vehicle Movement,Day of Week,Weekend,Time,Month,Year,Date in Month
0,Property Damage Crash,Maryland (State),GEORGIA AVE,ARCOLA AVE,CLEAR,DRY,DAWN,NONE DETECTED,-0.015398,2018,...,No,"CARGO VAN/LIGHT TRUCK 2 AXLES (OVER 10,000LBS ...",FREIGHTLINER,STOPPED IN TRAFFIC LANE,1,0,7,0,2020,21
1,Injury Crash,Maryland (State),WOODFIELD RD,WARFIELD RD,CLOUDY,DRY,DAYLIGHT,NONE DETECTED,-0.672483,2017,...,Yes,PASSENGER CAR,CHEV,MAKING LEFT TURN,4,0,13,11,2019,6
2,Property Damage Crash,Maryland (State),NORBECK RD,E GUDE DR,RAINING,WET,DAYLIGHT,NONE DETECTED,1.955855,2000,...,No,PASSENGER CAR,TOYOTA,MAKING LEFT TURN,2,0,15,2,2020,25
3,Injury Crash,County,SHADY GROVE RD,CRABBS BRANCH WAY,CLEAR,DRY,DARK LIGHTS ON,NONE DETECTED,0.641686,2015,...,No,"CARGO VAN/LIGHT TRUCK 2 AXLES (OVER 10,000LBS ...",INTL,MOVING CONSTANT SPEED,0,0,19,0,2017,9
4,Injury Crash,Maryland (State),GEORGIA AVE,ENT TO SHOPPING CENTER,RAINING,WET,DAYLIGHT,NONE DETECTED,-0.015398,2017,...,No,(SPORT) UTILITY VEHICLE,CHEVROLET,MOVING CONSTANT SPEED,3,0,11,3,2020,9


#### Check length

In [ ]:
print(f'Length of train df:{len(train_df)}')
print(f'Length of test df:{len(test_df)}')

Length of train df:74856
Length of test df:32082


#### Get full df to build the graph

In [ ]:
test_df[['ACRS Report Type']].value_counts()

ACRS Report Type     
Property Damage Crash    19283
Injury Crash             12707
Fatal Crash                 92
dtype: int64

In [ ]:
df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106938 entries, 0 to 106937
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ACRS Report Type        106938 non-null  object 
 1   Route Type              106938 non-null  object 
 2   Road Name               106938 non-null  object 
 3   Cross-Street Name       106938 non-null  object 
 4   Weather                 106938 non-null  object 
 5   Surface Condition       106938 non-null  object 
 6   Light                   106938 non-null  object 
 7   Driver Substance Abuse  106938 non-null  object 
 8   Speed Limit             106938 non-null  float64
 9   Vehicle Year            106938 non-null  int64  
 10  Latitude                106938 non-null  float64
 11  Longitude               106938 non-null  float64
 12  Traffic Control         106938 non-null  object 
 13  Collision Type          106938 non-null  object 
 14  Driver At Fault     

---

## Step I: Preprocess

#### Cat Features

***'Vehicle Body Type'***

In [ ]:
'''
There are several duplicated level due to the Upper Lower Case.
'''
df['Vehicle Body Type'].value_counts()

PASSENGER CAR                                                71947
(SPORT) UTILITY VEHICLE                                      10343
PICKUP TRUCK                                                  4104
Passenger Car                                                 3421
VAN                                                           3136
TRANSIT BUS                                                   2245
SCHOOL BUS                                                    1580
OTHER LIGHT TRUCKS (10,000LBS (4,536KG) OR LESS)              1114
CARGO VAN/LIGHT TRUCK 2 AXLES (OVER 10,000LBS (4,536 KG))     1058
MEDIUM/HEAVY TRUCKS 3 AXLES (OVER 10,000LBS (4,536KG))         908
POLICE VEHICLE/NON EMERGENCY                                   881
Sport Utility Vehicle                                          828
OTHER                                                          801
POLICE VEHICLE/EMERGENCY                                       670
MOTORCYCLE                                                    

In [ ]:
vehicle_type_mapping = {
    'PASSENGER CAR': 'Passenger Cars',
    'Passenger Car': 'Passenger Cars',
    'STATION WAGON': 'Passenger Cars',
    'Station Wagon': 'Passenger Cars',
    '(SPORT) UTILITY VEHICLE': 'SUVs',
    'Sport Utility Vehicle': 'SUVs',
    'PICKUP TRUCK': 'Pickup Trucks',
    'Pickup': 'Pickup Trucks',
    'VAN': 'Vans',
    'Van - Passenger (&lt;9 Seats)': 'Vans',
    'Van - Cargo': 'Vans',
    'TRANSIT BUS': 'Buses',
    'SCHOOL BUS': 'Buses',
    'Bus - Transit': 'Buses',
    'Bus - School': 'Buses',
    'OTHER BUS': 'Buses',
    'Bus - Other Type': 'Buses',
    'CROSS COUNTRY BUS': 'Buses',
    'MOTORCYCLE': 'Motorcycles and Mopeds',
    'Motorcycle - 2 Wheeled': 'Motorcycles and Mopeds',
    'MOPED': 'Motorcycles and Mopeds',
    'Moped Or motorized bicycle': 'Motorcycles and Mopeds',
    'OTHER LIGHT TRUCKS (10,000LBS (4,536KG) OR LESS)': 'Trucks',
    'CARGO VAN/LIGHT TRUCK 2 AXLES (OVER 10,000LBS (4,536 KG))': 'Trucks',
    'MEDIUM/HEAVY TRUCKS 3 AXLES (OVER 10,000LBS (4,536KG))': 'Trucks',
    'TRUCK TRACTOR': 'Trucks',
    'Truck Tractor': 'Trucks',
    'Single-Unit Truck': 'Trucks',
    'Other Trucks': 'Trucks',
    'POLICE VEHICLE/NON EMERGENCY': 'Emergency Vehicles',
    'POLICE VEHICLE/EMERGENCY': 'Emergency Vehicles',
    'AMBULANCE/EMERGENCY': 'Emergency Vehicles',
    'AMBULANCE/NON EMERGENCY': 'Emergency Vehicles',
    'FIRE VEHICLE/EMERGENCY': 'Emergency Vehicles',
    'FIRE VEHICLE/NON EMERGENCY': 'Emergency Vehicles',
    'RECREATIONAL VEHICLE': 'Recreational Vehicles',
    'SNOWMOBILE': 'Recreational Vehicles',
    'Snowmobile': 'Recreational Vehicles',
    'Recreational Off-Highway Vehicles (ROV)': 'Recreational Vehicles',
    'ALL TERRAIN VEHICLE (ATV)': 'Recreational Vehicles',
    'All-Terrain Vehicle/All-Terrain Cycle (ATV/ATC)': 'Recreational Vehicles',
    'FARM VEHICLE': 'Specialty Vehicles',
    'AUTOCYCLE': 'Specialty Vehicles',
    'LOW SPEED VEHICLE': 'Specialty Vehicles',
    'LIMOUSINE': 'Specialty Vehicles',
    'Construction Equipment (backhoe, bulldozer, etc.)': 'Specialty Vehicles',
    'UNKNOWN': 'Unknown',
    'Unknown': 'Unknown',
    'OTHER': 'Other',
    'Other': 'Other',
}

df['Vehicle Body Type'] = df['Vehicle Body Type'].map(vehicle_type_mapping).fillna('Other')

df['Vehicle Body Type'].value_counts()

Passenger Cars            75865
SUVs                      11171
Pickup Trucks              4362
Buses                      4242
Trucks                     3475
Vans                       3284
Emergency Vehicles         2165
Other                       834
Motorcycles and Mopeds      649
Unknown                     563
Recreational Vehicles       271
Specialty Vehicles           57
Name: Vehicle Body Type, dtype: int64

***'Route Type'***

In [ ]:
df['Route Type'].value_counts()

Maryland (State)          51531
County                    36333
Municipality               5930
US (State)                 4865
County Route               2367
Interstate (State)         1873
Maryland (State) Route     1605
Other Public Roadway        831
Ramp                        528
Municipality Route          516
Government                  338
Local Route                 143
Government Route             33
Service Road                 28
Unknown                       8
Crossover                     6
Private Route                 3
Name: Route Type, dtype: int64

In [ ]:
location_type_mapping = {
    'Maryland (State)': 'State Roads',
    'US (State)': 'State Roads',
    'Interstate (State)': 'State Roads',
    'Maryland (State) Route': 'State Roads',
    'County': 'County Roads',
    'County Route': 'County Roads',
    'Municipality': 'Municipality Roads',
    'Municipality Route': 'Municipality Roads',
    'Other Public Roadway': 'Other Public Roadways',
    'Local Route': 'Other Public Roadways',
    'Ramp': 'Other Public Roadways',
    'Service Road': 'Other Public Roadways',
    'Crossover': 'Other Public Roadways',
    'Government': 'Government Roads',
    'Government Route': 'Government Roads',
    'Private Route': 'Private Roads',
    'Unknown': 'Unknown',
}

df['Route Type'] = df['Route Type'].map(location_type_mapping)
df['Route Type'].value_counts()

State Roads              59874
County Roads             38700
Municipality Roads        6446
Other Public Roadways     1536
Government Roads           371
Unknown                      8
Private Roads                3
Name: Route Type, dtype: int64

***'Weather'***

In [ ]:
df['Weather'].value_counts()

CLEAR                                74144
RAINING                              13737
CLOUDY                               11740
Clear                                 3919
SNOW                                   882
Rain                                   598
Cloudy                                 467
FOGGY                                  455
WINTRY MIX                             233
UNKNOWN                                207
OTHER                                  193
SLEET                                  127
SEVERE WINDS                           100
BLOWING SNOW                            71
Snow                                    23
Blowing Snow                            16
Freezing Rain Or Freezing Drizzle        7
Fog, Smog, Smoke                         7
Severe Crosswinds                        6
BLOWING SAND, SOIL, DIRT                 5
Unknown                                  1
Name: Weather, dtype: int64

In [ ]:
weather_condition_mapping = {
    'CLEAR': 'Clear',
    'Clear': 'Clear',
    'RAINING': 'Rain',
    'Rain': 'Rain',
    'Freezing Rain Or Freezing Drizzle': 'Rain',
    'CLOUDY': 'Cloudy',
    'Cloudy': 'Cloudy',
    'SNOW': 'Snow',
    'Snow': 'Snow',
    'BLOWING SNOW': 'Snow',
    'Blowing Snow': 'Snow',
    'FOGGY': 'Foggy',
    'Fog, Smog, Smoke': 'Foggy',
    'WINTRY MIX': 'Wintry Mix',
    'SLEET': 'Wintry Mix',
    'SEVERE WINDS': 'Severe Winds',
    'Severe Crosswinds': 'Severe Winds',
    'BLOWING SAND, SOIL, DIRT': 'Severe Winds',
    'UNKNOWN': 'Unknown',
    'Unknown': 'Unknown',
    'OTHER': 'Other',
}

df['Weather'] = df['Weather'].map(weather_condition_mapping)
df['Weather'].value_counts()

Clear           78063
Rain            14342
Cloudy          12207
Snow              992
Foggy             462
Wintry Mix        360
Unknown           208
Other             193
Severe Winds      111
Name: Weather, dtype: int64

***'Surface Condition'***

In [ ]:
df['Surface Condition'].value_counts()

DRY                         80332
WET                         19832
Dry                          4210
Wet                           781
ICE                           615
SNOW                          596
UNKNOWN                       216
SLUSH                         122
OTHER                         100
MUD, DIRT, GRAVEL              30
WATER(STANDING/MOVING)         26
OIL                            21
Snow                           19
Ice/Frost                      12
Slush                          12
Other                           8
SAND                            4
Water (standing, moving)        2
Name: Surface Condition, dtype: int64

In [ ]:
surface_condition_mapping = {
    'DRY': 'Dry',
    'Dry': 'Dry',
    'WET': 'Wet',
    'Wet': 'Wet',
    'SNOW': 'Snow',
    'Snow': 'Snow',
    'ICE': 'Ice',
    'Ice/Frost': 'Ice',
    'SLUSH': 'Slush',
    'Slush': 'Slush',
    'MUD, DIRT, GRAVEL': 'Loose Material',
    'SAND': 'Loose Material',
    'WATER(STANDING/MOVING)': 'Water',
    'Water (standing, moving)': 'Water',
    'OIL': 'Oil',
    'UNKNOWN': 'Unknown',
    'OTHER': 'Other',
    'Other': 'Other',
}


df['Surface Condition'] = df['Surface Condition'].map(surface_condition_mapping)
df['Surface Condition'].value_counts()

Dry               84542
Wet               20613
Ice                 627
Snow                615
Unknown             216
Slush               134
Other               108
Loose Material       34
Water                28
Oil                  21
Name: Surface Condition, dtype: int64

***'Light'***

In [ ]:
df['Light'].value_counts()

DAYLIGHT                    70454
DARK LIGHTS ON              23116
Daylight                     3596
DARK NO LIGHTS               2913
DUSK                         2253
DAWN                         2081
Dark - Lighted               1175
DARK -- UNKNOWN LIGHTING      791
OTHER                         147
UNKNOWN                       139
Dark - Not Lighted            109
Dusk                           69
Dawn                           60
Dark - Unknown Lighting        19
Other                          14
Unknown                         2
Name: Light, dtype: int64

In [ ]:
light_mapping = {
    'DAYLIGHT': 'Daylight',
    'Daylight': 'Daylight',
    'DARK LIGHTS ON': 'Dark - Lighted',
    'Dark - Lighted': 'Dark - Lighted',
    'DARK NO LIGHTS': 'Dark - Not Lighted',
    'Dark - Not Lighted': 'Dark - Not Lighted',
    'DARK -- UNKNOWN LIGHTING': 'Dark - Unknown Lighting',
    'Dark - Unknown Lighting': 'Dark - Unknown Lighting',
    'DUSK': 'Dusk',
    'Dusk': 'Dusk',
    'DAWN': 'Dawn',
    'Dawn': 'Dawn',
    'OTHER': 'Other',
    'Other': 'Other',
    'UNKNOWN': 'Unknown',
    'Unknown': 'Unknown',
}

df['Light'] = df['Light'].map(light_mapping)
df['Light'].value_counts()

Daylight                   74050
Dark - Lighted             24291
Dark - Not Lighted          3022
Dusk                        2322
Dawn                        2141
Dark - Unknown Lighting      810
Other                        161
Unknown                      141
Name: Light, dtype: int64

In [ ]:
df['Light'].value_counts()

Daylight                   74050
Dark - Lighted             24291
Dark - Not Lighted          3022
Dusk                        2322
Dawn                        2141
Dark - Unknown Lighting      810
Other                        161
Unknown                      141
Name: Light, dtype: int64

***'Driver Substance Abuse'***

In [ ]:
'''
Present = The drug was detected but its role or impact is not specified.
Contributed = The drug was involved in influencing or causing an event.
'''
df['Driver Substance Abuse'].value_counts()

NONE DETECTED                                          92454
UNKNOWN                                                 5375
Not Suspect of Alcohol Use, Not Suspect of Drug Use     4734
ALCOHOL PRESENT                                         2617
ALCOHOL CONTRIBUTED                                      977
Unknown, Unknown                                         186
ILLEGAL DRUG PRESENT                                     164
Suspect of Alcohol Use, Not Suspect of Drug Use           92
MEDICATION PRESENT                                        76
ILLEGAL DRUG CONTRIBUTED                                  75
COMBINED SUBSTANCE PRESENT                                57
MEDICATION CONTRIBUTED                                    41
COMBINATION CONTRIBUTED                                   30
OTHER                                                     28
Suspect of Alcohol Use, Unknown                            8
Not Suspect of Alcohol Use, Unknown                        7
Unknown, Not Suspect of 

In [ ]:
substance_abuse_mapping = {
    'NONE DETECTED': 'None Detected',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'None Detected',
    'ALCOHOL PRESENT': 'Alcohol Present',
    'Suspect of Alcohol Use, Not Suspect of Drug Use': 'Alcohol Present',
    'Suspect of Alcohol Use, Unknown': 'Alcohol Present',
    'ALCOHOL CONTRIBUTED': 'Alcohol Contributed',
    'ILLEGAL DRUG PRESENT': 'Drug Present',
    'ILLEGAL DRUG CONTRIBUTED': 'Drug Contributed',
    'MEDICATION PRESENT': 'Medication Present',
    'MEDICATION CONTRIBUTED': 'Medication Contributed',
    'COMBINED SUBSTANCE PRESENT': 'Combined Substances Present',
    'COMBINATION CONTRIBUTED': 'Combined Substances Contributed',
    'UNKNOWN': 'Unknown',
    'Unknown, Unknown': 'Unknown',
    'Unknown, Not Suspect of Drug Use': 'Unknown',
    'Not Suspect of Alcohol Use, Unknown': 'Unknown',
    'OTHER': 'Other',
}

df['Driver Substance Abuse'] = df['Driver Substance Abuse'].map(substance_abuse_mapping).fillna('Other')
df['Driver Substance Abuse'].value_counts()

None Detected                      97188
Unknown                             5574
Alcohol Present                     2717
Alcohol Contributed                  977
Drug Present                         164
Medication Present                    76
Drug Contributed                      75
Combined Substances Present           57
Medication Contributed                41
Other                                 39
Combined Substances Contributed       30
Name: Driver Substance Abuse, dtype: int64

***'Collision Type'***

In [ ]:
df['Collision Type'].value_counts()

SAME DIR REAR END                35917
STRAIGHT MOVEMENT ANGLE          20543
SAME DIRECTION SIDESWIPE          9444
HEAD ON LEFT TURN                 9066
SINGLE VEHICLE                    8017
OTHER                             6709
SAME DIRECTION RIGHT TURN         2266
SAME DIRECTION LEFT TURN          2236
HEAD ON                           2105
Angle                             1704
ANGLE MEETS LEFT TURN             1405
OPPOSITE DIRECTION SIDESWIPE      1365
Front to Rear                     1055
ANGLE MEETS RIGHT TURN             786
Sideswipe, Same Direction          481
Rear To Side                       465
SAME DIR REND RIGHT TURN           456
SAME DIR BOTH LEFT TURN            449
ANGLE MEETS LEFT HEAD ON           436
SAME DIR REND LEFT TURN            412
Other                              398
Front to Front                     394
Single Vehicle                     315
Sideswipe, Opposite Direction      189
OPPOSITE DIR BOTH LEFT TURN        178
UNKNOWN                  

In [ ]:
collision_mapping = {
    'SAME DIR REAR END': 'Same Direction Rear-End',
    'SAME DIR REND RIGHT TURN': 'Same Direction Rear-End',
    'SAME DIR REND LEFT TURN': 'Same Direction Rear-End',
    'STRAIGHT MOVEMENT ANGLE': 'Angle Collisions',
    'ANGLE MEETS LEFT TURN': 'Angle Collisions',
    'ANGLE MEETS RIGHT TURN': 'Angle Collisions',
    'Angle': 'Angle Collisions',
    'SAME DIRECTION SIDESWIPE': 'Same Direction Sideswipe',
    'Sideswipe, Same Direction': 'Same Direction Sideswipe',
    'HEAD ON LEFT TURN': 'Opposite Direction Collisions',
    'HEAD ON': 'Opposite Direction Collisions',
    'OPPOSITE DIRECTION SIDESWIPE': 'Opposite Direction Collisions',
    'Front to Front': 'Opposite Direction Collisions',
    'SINGLE VEHICLE': 'Single Vehicle',
    'Single Vehicle': 'Single Vehicle',
    'SAME DIRECTION RIGHT TURN': 'Turn Collisions',
    'SAME DIRECTION LEFT TURN': 'Turn Collisions',
    'SAME DIR BOTH LEFT TURN': 'Turn Collisions',
    'OPPOSITE DIR BOTH LEFT TURN': 'Turn Collisions',
    'OTHER': 'Other',
    'Other': 'Other',
    'UNKNOWN': 'Unknown',
    'Unknown': 'Unknown',
    'Rear To Rear': 'Other',  # I don't understand ???????
}


df['Collision Type'] = df['Collision Type'].map(collision_mapping).fillna('Other')
df['Collision Type'].value_counts()

Same Direction Rear-End          36785
Angle Collisions                 24438
Opposite Direction Collisions    12930
Same Direction Sideswipe          9925
Other                             9273
Single Vehicle                    8332
Turn Collisions                   5129
Unknown                            126
Name: Collision Type, dtype: int64

***'Traffic Control'***

In [ ]:
df['Traffic Control'].value_counts()

NO CONTROLS                                                                 44871
TRAFFIC SIGNAL                                                              43605
STOP SIGN                                                                    8868
Traffic Control Signal                                                       2852
FLASHING TRAFFIC SIGNAL                                                      1525
OTHER                                                                        1310
YIELD SIGN                                                                   1257
No Controls                                                                  1243
Stop Sign                                                                     598
PERSON                                                                        193
Flashing Traffic Control Signal                                               176
WARNING SIGN                                                                  116
UNKNOWN         

In [ ]:
control_mapping = {
    'NO CONTROLS': 'No Controls',
    'No Controls': 'No Controls',
    'TRAFFIC SIGNAL': 'Traffic Signal',
    'Traffic Control Signal': 'Traffic Signal',
    'Flashing Traffic Control Signal': 'Traffic Signal',
    'FLASHING TRAFFIC SIGNAL': 'Traffic Signal',
    'Other Signal': 'Traffic Signal',
    'STOP SIGN': 'Stop Sign',
    'Stop Sign': 'Stop Sign',
    'YIELD SIGN': 'Yield Sign',
    'Yield Sign': 'Yield Sign',
    'PERSON': 'Pedestrian/Crossing Control',
    'Pedestrian Crossing Sign': 'Pedestrian/Crossing Control',
    'Pedestrian Crossing': 'Pedestrian/Crossing Control',
    'Person (including flagger, law enforcement, crossing guard, etc.)': 'Pedestrian/Crossing Control',
    'RAILWAY CROSSING DEVICE': 'Railway Crossing',
    'Flashing Railroad Crossing Signal (may include gates)': 'Railway Crossing',
    'WARNING SIGN': 'Warning Sign',
    'Intersection Ahead Warning Sign': 'Warning Sign',
    'Other Warning Sign': 'Warning Sign',
    'School Zone Sign': 'Warning Sign',
    'SCHOOL ZONE SIGN DEVICE': 'Warning Sign',
    'OTHER': 'Other',
    'Other': 'Other',
    'Other Pavement Marking (excluding edgelines, centerlines, or lane lines)': 'Other',
    'UNKNOWN': 'Unknown',
}

df['Traffic Control'] = df['Traffic Control'].map(control_mapping).fillna('Other')
df['Traffic Control'].value_counts()

Traffic Signal                 48164
No Controls                    46114
Stop Sign                       9466
Other                           1388
Yield Sign                      1288
Pedestrian/Crossing Control      245
Warning Sign                     133
Unknown                          111
Railway Crossing                  29
Name: Traffic Control, dtype: int64

***'Vehicle Make'***

In [ ]:
'''
Too many, ignore this feature
'''
df['Vehicle Make'].value_counts()

TOYOTA        15051
HONDA         12404
FORD          10330
NISSAN         5522
TOYT           5478
              ...  
MER-BENZ          1
BMV               1
HYUNDY            1
GILLLIG BU        1
VOLS              1
Name: Vehicle Make, Length: 1370, dtype: int64

***'Vehicle Movement'***

In [ ]:
df['Vehicle Movement'].value_counts()

MOVING CONSTANT SPEED      41465
SLOWING OR STOPPING        15536
STOPPED IN TRAFFIC LANE    12091
MAKING LEFT TURN           11569
ACCELERATING                5353
MAKING RIGHT TURN           3339
STARTING FROM LANE          3116
CHANGING LANES              2942
Moving Constant Speed       2115
UNKNOWN                     1107
Turning Left                1078
BACKING                     1025
MAKING U TURN                791
ENTERING TRAFFIC LANE        512
PARKED                       500
PASSING                      492
STARTING FROM PARKED         480
SKIDDING                     479
Accelerating                 421
Slowing or Stopping          412
Stopped in Traffic           390
NEGOTIATING A CURVE          319
OTHER                        309
Turning Right                237
RIGHT TURN ON RED            188
PARKING                      144
LEAVING TRAFFIC LANE         126
Changing Lanes               114
Entering Traffic Lane        105
Making U-Turn                 53
Overtaking

In [ ]:
movement_mapping = {
    'MOVING CONSTANT SPEED': 'Constant Speed',
    'Moving Constant Speed': 'Constant Speed',
    'SLOWING OR STOPPING': 'Slowing or Stopping',
    'Slowing or Stopping': 'Slowing or Stopping',
    'STOPPED IN TRAFFIC LANE': 'Slowing or Stopping',
    'Stopped in Traffic': 'Slowing or Stopping',
    'MAKING LEFT TURN': 'Making Left Turn',
    'Turning Left': 'Making Left Turn',
    'MAKING RIGHT TURN': 'Making Right Turn',
    'Turning Right': 'Making Right Turn',
    'RIGHT TURN ON RED': 'Making Right Turn',
    'MAKING U TURN': 'Making U-Turn',
    'Making U-Turn': 'Making U-Turn',
    'ACCELERATING': 'Accelerating',
    'Accelerating': 'Accelerating',
    'CHANGING LANES': 'Changing Lanes',
    'Changing Lanes': 'Changing Lanes',
    'ENTERING TRAFFIC LANE': 'Entering/Leaving Traffic Lane',
    'Entering Traffic Lane': 'Entering/Leaving Traffic Lane',
    'LEAVING TRAFFIC LANE': 'Entering/Leaving Traffic Lane',
    'Leaving Traffic Lane': 'Entering/Leaving Traffic Lane',
    'STARTING FROM LANE': 'Starting Vehicle',
    'STARTING FROM PARKED': 'Starting Vehicle',
    'PARKED': 'Parking or Backing',
    'PARKING': 'Parking or Backing',
    'BACKING': 'Parking or Backing',
    'Backing': 'Parking or Backing',
    'PASSING': 'Passing/Overtaking',
    'Overtaking/Passing': 'Passing/Overtaking',
    'SKIDDING': 'Skidding/Negotiating Curve',
    'NEGOTIATING A CURVE': 'Skidding/Negotiating Curve',
    'Negotiating a Curve': 'Skidding/Negotiating Curve',
    'DRIVERLESS MOVING VEH.': 'Driverless Vehicle',
    'UNKNOWN': 'Other/Unknown',
    'OTHER': 'Other/Unknown',
}

df['Vehicle Movement'] = df['Vehicle Movement'].map(movement_mapping).fillna('Other/Unknown')
df['Vehicle Movement'].value_counts()

Constant Speed                   43580
Slowing or Stopping              28429
Making Left Turn                 12647
Accelerating                      5774
Making Right Turn                 3764
Starting Vehicle                  3596
Changing Lanes                    3056
Parking or Backing                1698
Other/Unknown                     1441
Making U-Turn                      844
Skidding/Negotiating Curve         816
Entering/Leaving Traffic Lane      752
Passing/Overtaking                 530
Driverless Vehicle                  11
Name: Vehicle Movement, dtype: int64

***'Driver At Fault'***

In [ ]:
df['Driver At Fault'].value_counts()

Yes        54422
No         50044
Unknown     2472
Name: Driver At Fault, dtype: int64

***Lower all cat features***

In [ ]:
cat_columns = df.select_dtypes(include=['object']).columns

for col in cat_columns:
    df[col] = df[col].str.lower()

In [ ]:
categorical_columns = df.select_dtypes(include='object').columns  # Select string (object) columns
df[categorical_columns] = df[categorical_columns].apply(lambda col: col.str.lower())

for col in categorical_columns:
    print(f"Value counts for column '{col}':")
    print(df[col].value_counts())
    print("\n")

Value counts for column 'ACRS Report Type':
property damage crash    64276
injury crash             42355
fatal crash                307
Name: ACRS Report Type, dtype: int64


Value counts for column 'Route Type':
state roads              59874
county roads             38700
municipality roads        6446
other public roadways     1536
government roads           371
unknown                      8
private roads                3
Name: Route Type, dtype: int64


Value counts for column 'Road Name':
georgia ave                                 7226
new hampshire ave                           4035
frederick rd                                3961
rockville pike                              3339
connecticut ave                             2988
                                            ... 
pepperwood la                                  1
milestone manor la                             1
fontaine st                                    1
ramp 3 fr shady grove rd eb to is 270 nb       1
ramp 1 fr

In [ ]:
df.head()

,ACRS Report Type,Route Type,Road Name,Cross-Street Name,Weather,Surface Condition,Light,Driver Substance Abuse,Speed Limit,Vehicle Year,...,Driver At Fault,Vehicle Body Type,Vehicle Make,Vehicle Movement,Day of Week,Weekend,Time,Month,Year,Date in Month
0,property damage crash,state roads,georgia ave,arcola ave,clear,dry,dawn,none detected,-0.015398,2018,...,no,trucks,freightliner,slowing or stopping,1,0,7,0,2020,21
1,injury crash,state roads,woodfield rd,warfield rd,cloudy,dry,daylight,none detected,-0.672483,2017,...,yes,passenger cars,chev,making left turn,4,0,13,11,2019,6
2,property damage crash,state roads,norbeck rd,e gude dr,rain,wet,daylight,none detected,1.955855,2000,...,no,passenger cars,toyota,making left turn,2,0,15,2,2020,25
3,injury crash,county roads,shady grove rd,crabbs branch way,clear,dry,dark - lighted,none detected,0.641686,2015,...,no,trucks,intl,constant speed,0,0,19,0,2017,9
4,injury crash,state roads,georgia ave,ent to shopping center,rain,wet,daylight,none detected,-0.015398,2017,...,no,suvs,chevrolet,constant speed,3,0,11,3,2020,9


---

---

## Feature Engineering

In [ ]:
cat_features = [
    'Weather', 'Surface Condition', 'Light', 'Driver Substance Abuse',
    'Traffic Control', 'Collision Type', 'Vehicle Movement', 'Route Type',
    'Vehicle Body Type', 'Driver At Fault'
]

In [ ]:
num_features = [
    'Day of Week',
    'Weekend',
    'Time',
    'Month',
    'Year',
    'Date in Month',
    'Speed Limit',
    'Latitude',
    'Longitude'
]

In [ ]:
# Encode target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['ACRS Report Type'])

In [ ]:
# Save the label encoding for later interpretation
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [ ]:
label_mapping

{'fatal crash': 0, 'injury crash': 1, 'property damage crash': 2}

In [ ]:
# Separate features and target
X = df[cat_features + num_features]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['ACRS Report Type'])

# Define preprocessing pipelines
cat_transformer = OneHotEncoder(handle_unknown='ignore')
num_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)

In [ ]:
X_preprocessed = preprocessor.fit_transform(X)

In [ ]:
X_train = X_preprocessed[:-32082]
X_test = X_preprocessed[-32082:]
y_train = y[:-32082]
y_test = y[-32082:]
print(X_train.shape)
print(X_test.shape)

(74856, 100)
(32082, 100)


In [ ]:
assert X_train.shape[0] == len(y_train)

In [ ]:
# Preprocess
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

---

---

## Model Tuning

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Split X_train into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

# Ensure dense format
if issparse(X_train):
    X_train = X_train.toarray()
if issparse(X_val):
    X_val = X_val.toarray()

In [ ]:
class TabularTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, d_model, nhead, num_layers, dropout):
        super(TabularTransformer, self).__init__()
        self.input_projection = nn.Linear(input_dim, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, input_dim, d_model))

        transformer_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)

        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.input_projection(x)
        x = self.transformer(x.unsqueeze(1))
        x = x.mean(dim=1)
        x = self.fc(x)

        return x

In [ ]:
def objective(trial):
    nhead = trial.suggest_int("nhead", 2, 8, step=2)
    d_model = trial.suggest_int("d_model", 16, 128, step=16)
    if d_model % nhead != 0:
        d_model = (d_model // nhead) * nhead

    # Hyperparameters
    num_layers = trial.suggest_int("num_layers", 1, 4)
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.1)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)

    # Model
    model = TabularTransformer(
        input_dim=X_train.shape[1],
        num_classes=len(set(y_train)),
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    # Loss & Opt
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Data Loader
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.float32).to(device),
        torch.tensor(y_train, dtype=torch.long).to(device)
    )
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    val_dataset = TensorDataset(
        torch.tensor(X_val, dtype=torch.float32).to(device),
        torch.tensor(y_val, dtype=torch.long).to(device)
    )
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Train
    model.train()
    for epoch in range(10):  # Short epochs for tuning
        for x, y in train_loader:
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

    # Val
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in val_loader:
            outputs = model(x)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, output_dict=True)
    f1 = report['weighted avg']['f1-score']

    print(f"Confusion Matrix:\n{cm}")
    print(f"Classification Report:\n{classification_report(all_labels, all_preds)}")

    return f1

In [ ]:
tuner = False

In [ ]:
if tuner == True:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    # Best trial
    print("Best trial:", study.best_trial.params)

---

---

***Rebuild the model with best params***

In [ ]:
best_params_store = {'nhead': 4, 'd_model': 32, 'num_layers': 1, 'dropout': 0.1, 'lr': 0.001673600994360086}

In [ ]:
# best_params_test= {'nhead': 4, 'd_model': 32, 'num_layers': 1, 'dropout': 0.1, 'lr': 0.001673600994360086}

In [ ]:
# Best hyperparameters
best_params = best_params_store

best_model = TabularTransformer(
    input_dim=X_test.shape[1],
    num_classes=3,
    d_model=best_params['d_model'],
    nhead=best_params['nhead'],
    num_layers=best_params['num_layers'],
    dropout=best_params['dropout']
).to(device)

/Users/yunxianghan/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Data Loader
train_dataset = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32).to(device),
    torch.tensor(y_train, dtype=torch.long).to(device)
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
# class_weights = torch.tensor([0.2, 0.4, 0.4])
# class_weights = torch.tensor([0.15, 2, 2])
learning_rate = 0.001673600994360086
# criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
criterion = nn.CrossEntropyLoss()

In [ ]:
# Set the number of epochs
num_epochs = 10
optimizer = torch.optim.Adam(best_model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0.0
    correct = 0
    total = 0

    for batch in train_loader:
        X_batch, y_batch = batch
        outputs = best_model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * X_batch.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    epoch_loss /= total
    accuracy = correct / total * 100
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch [1/10], Loss: 0.5369, Accuracy: 70.86%
Epoch [2/10], Loss: 0.4565, Accuracy: 73.86%
Epoch [3/10], Loss: 0.4443, Accuracy: 74.59%
Epoch [4/10], Loss: 0.4359, Accuracy: 75.04%
Epoch [5/10], Loss: 0.4301, Accuracy: 75.43%
Epoch [6/10], Loss: 0.4256, Accuracy: 75.73%
Epoch [7/10], Loss: 0.4215, Accuracy: 76.06%
Epoch [8/10], Loss: 0.4164, Accuracy: 76.46%
Epoch [9/10], Loss: 0.4120, Accuracy: 76.98%
Epoch [10/10], Loss: 0.4071, Accuracy: 77.35%


---

---

***Make prediction on test***

In [ ]:
if issparse(X_test):
    X_test = X_test.toarray()

In [ ]:
best_model.eval()

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

with torch.no_grad():
    outputs = best_model(X_test_tensor)
    _, preds = torch.max(outputs, 1)

preds = preds.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()

In [ ]:
accuracy = accuracy_score(y_test_np, preds)
precision = precision_score(y_test_np, preds, average='weighted')
recall = recall_score(y_test_np, preds, average='weighted')
f1 = f1_score(y_test_np, preds, average='weighted')

print(f"Accuracy: {accuracy * 100:.6f}%")
print(f"Precision: {precision * 100:.6f}%")
print(f"Recall: {recall * 100:.6f}%")
print(f"F1 Score: {f1 * 100:.6f}%")

Accuracy: 75.015990%
Precision: 76.707789%
Recall: 75.015990%
F1 Score: 73.858543%


---

***Check overfitting***

In [ ]:
best_model.eval()
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

with torch.no_grad():
    train_outputs = best_model(X_train_tensor)
    _, preds = torch.max(train_outputs, 1)

train_preds = preds.cpu().numpy()
y_train_np = y_train_tensor.cpu().numpy()

In [ ]:
accuracy = accuracy_score(y_train_np, train_preds)
precision = precision_score(y_train_np, train_preds, average='weighted')
recall = recall_score(y_train_np, train_preds, average='weighted')
f1 = f1_score(y_train_np, train_preds, average='weighted')

print(f"Accuracy: {accuracy * 100:.6f}%")
print(f"Precision: {precision * 100:.6f}%")
print(f"Recall: {recall * 100:.6f}%")
print(f"F1 Score: {f1 * 100:.6f}%")

Accuracy: 77.870591%
Precision: 80.161895%
Recall: 77.870591%
F1 Score: 76.882550%


---

---